In [23]:
import pandas as pd
import plotly.express as px

## Finding the ISS

In [24]:
# Reading in json API
url = 'http://api.open-notify.org/iss-now.json'
df = pd.read_json(url)

In [25]:
# Wranging the data into the needed format
df['latitude'] = df.loc['latitude', 'iss_position']
df['longitude'] = df.loc['longitude', 'iss_position']
df.reset_index(inplace=True)
df = df.drop(['index', 'message'], axis = 1)
df

,iss_position,timestamp,latitude,longitude
0,-48.2589,2022-04-03 15:01:56,-48.2589,171.0391
1,171.0391,2022-04-03 15:01:56,-48.2589,171.0391


In [26]:
# Plotting the current position
figure = px.scatter_geo(df, lat='latitude', lon='longitude')
figure.show()